# CS505_Final_Project
Handi Xie (handi.xie.beintouch@gmail.com), Zhichen Chen(zhichenc@bu.edu)
# Setup

In [ ]:
!pip install praw

In [ ]:
!rm -rf /content/drive/MyDrive/BU/CS505/data_dump/partial_reddit*

In [ ]:
import json
import praw

# Path to the JSON file
auth_path = "/content/drive/MyDrive/Colab Notebooks/auth/client_secrets.json"

# Read the JSON file
with open(auth_path, 'r') as file:
    auth_data = json.load(file)

# Extract credentials
client_id = auth_data['client_id']
client_secret = auth_data['clients_secrets']
user_agent = auth_data['user_agent']
redirect_url = auth_data['redirect_url']
password=auth_data['password']
username=auth_data['username']

# Initialize the Reddit client
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent,
                     redirect_url=redirect_url,
                     password=password,
                     username=username,
                     ratelimit_seconds=300,
                     check_for_async=False)

# Now you can use 'reddit' to interact with the Reddit API
# For example, you can print the name of the subreddit
print(reddit.subreddit("news").display_name)

news


1. worldnews
2. news
3. UpliftingNews
4. politics
5. WhitePeopleTwitter
6. China
7. LOOK_CHINA
8. China_irl
9. real_China_irl
10. DoubanGoosegroup


# Global Variables

In [ ]:
# Define the subreddits to traverse
# subreddits = ["worldnews", "news"]
# Define the subreddits to traverse
subreddits = ["worldnews", "news", "politics",
              "China", "China_irl", "real_China_irl"]

# subreddits = ["news"]

# Define the date range in UTC
start_date = int(datetime.datetime(2023, 12, 11).timestamp())
end_date = int(datetime.datetime(2023, 12, 18).timestamp())

# Define a limit for each subreddit
limit = 2000

abs_path = "/content/drive/MyDrive/BU/CS505/data_dump"

processed_submissions = {}

In [ ]:
import datetime
from tqdm import tqdm
import praw
import pandas as pd

# Function to determine content type
def get_content_type(submission):
    if submission.is_self:
        return 'text'
    elif submission.is_video:
        return 'video'
    elif submission.is_reddit_media_domain:
        return 'image'
    else:
        return 'link'


# Function to collect posts and top/bottom 10 comments in a date range
def collect_posts_and_top_bottom_comments(subreddits, limit, start_date, end_date, processed_submissions):
    all_data = []
    for subreddit in subreddits:
        for submission in tqdm(reddit.subreddit(subreddit).new(limit=limit),
                               total=limit,
                               desc=f"Processing {limit} posts from /r/{subreddit}"):

            submission_temp = submission

            if submission.id in processed_submissions:
                continue

            try:
                if start_date <= submission.created_utc <= end_date:
                    try:
                        post_title = submission.title if submission.title else "Null"
                    except AttributeError:
                        post_title = "Null"
                    try:
                        poster_karma = submission.author.link_karma if submission.author else -1
                    except AttributeError:
                        poster_karma = -1

                    submission.comments.replace_more(limit=0)  # Remove MoreComments
                    comments = submission.comments.list()
                    comments.sort(key=lambda comment: comment.score, reverse=True)  # Sort by score

                    # Get top 3 comments
                    top_comments = comments[:3]
                    # bottom_comments = comments[-10:] if len(comments) > 10 else []

                    # Combine and remove duplicates
                    # unique_comments = {comment.id: comment for comment in top_comments + bottom_comments}.values()
                    unique_comments = top_comments

                    # Initlize post and comment block
                    post_and_comment_data = {}

                    for comment in unique_comments:
                        try:
                            commenter_karma = comment.author.comment_karma if comment.author else -1
                        except AttributeError:
                            commenter_karma = -1

                        post_and_comment_data = {
                            'post_title': post_title,
                            'post_date': datetime.datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                            'post_link': submission.permalink,
                            'post_content': submission.selftext if submission.is_self else submission.url,
                            'post_content_type': get_content_type(submission),
                            'post_upvote_count': submission.score,
                            'post_comment_count': submission.num_comments,
                            # 'upvote_ratio' : submission.upvote_ratio,
                            'poster_karma': poster_karma,
                            'commenter_username': comment.author.name if comment.author else 'Deleted',
                            'commenter_karma': commenter_karma,
                            'comment_text': comment.body,
                            'comment_upvotes': comment.score
                        }
                        all_data.append(post_and_comment_data)

                    # debug
                    if submission:
                        print(submission)
                    if post_and_comment_data:
                        print("Subreddit: {}, post link: {}, post date: {}".format(str(subreddit),
                                                                                post_and_comment_data['post_link'],
                                                                                post_and_comment_data['post_date']))

                    # Add submission id to the processed set
                    processed_submissions.add(submission.id)

                    # Save data to a file periodically
                    df = pd.DataFrame(all_data)
                    df.to_csv('{}/partial_reddit_data_{}.csv'.format(abs_path, submission.id), index=False)

            except praw.exceptions.PRAWException:
                print(f"Resource not found. Skipping /r/{subreddit} or submission.")
                continue

            except praw.exceptions.RedditAPIException as e:
                print(f"Rate limit exceeded. Sleeping for {e.sleep_time} seconds.")
                time.sleep(e.sleep_time)

            except Exception as e:  # Catching all other exceptions
                print(f"An unexpected error occurred: {e}")
                continue

    return all_data

In [ ]:
# Collect posts and comments
collected_data = collect_posts_and_top_bottom_comments(subreddits, limit, start_date=start_date,
                                                       end_date=end_date, processed_submissions=processed_submissions)

# Create a DataFrame from the collected data
df_results = pd.DataFrame(collected_data)

Processing 2000 posts from /r/China:  12%|█▏        | 246/2000 [00:03<00:17, 102.93it/s]

An unexpected error occurred: received 404 HTTP response


Processing 2000 posts from /r/China_irl:  17%|█▋        | 339/2000 [00:06<00:30, 53.92it/s]

18kdjpp
Subreddit: China_irl, post link: /r/China_irl/comments/18kdjpp/从以色列击杀人质事件是不是可以看出哈马斯还有可观的战力/, post date: 2023-12-17 09:19:46


Processing 2000 posts from /r/China_irl:  17%|█▋        | 345/2000 [00:07<00:38, 42.91it/s]

...

Processing 2000 posts from /r/real_China_irl:  32%|███▏      | 640/2000 [00:10<00:21, 63.81it/s]

18k5sak
Subreddit: real_China_irl, post link: /r/real_China_irl/comments/18k5sak/董志民思想看样子已经流传到民间了一个女主持发视频挺董宇辉弹幕里都让她嫁给他生八个/, post date: 2023-12-17 01:19:51


Processing 2000 posts from /r/real_China_irl:  39%|███▊      | 773/2000 [00:12<00:20, 60.73it/s]

18izy2l
Subreddit: real_China_irl, post link: /r/real_China_irl/comments/18izy2l/只能怪她妈不长眼嫁给国男了欧美父亲抚养费可是强制的但愿她以后不会再嫁国男让这种悲剧重演/, post date: 2023-12-15 13:21:05


Processing 2000 posts from /r/real_China_irl:  49%|████▉     | 986/2000 [00:15<00:15, 63.61it/s]


In [ ]:
print(len(processed_submissions))

1764


In [ ]:
import pandas as pd
import glob

def concat_csv(abs_path):
    # Concatenate CSV files and remove duplicates
    file_pattern = abs_path + "/*.csv"
    csv_files = glob.glob(file_pattern)
    df_final = pd.concat((pd.read_csv(f, encoding='utf8') for f in csv_files))
    df_final.drop_duplicates(inplace=True)

    # Save the cleaned DataFrame
    output_file_path = abs_path + "/combined.csv"
    df_final.to_csv(output_file_path, index=False, encoding='utf_8_sig')

    return df_final

In [ ]:
# Usage
abs_path = "/content/drive/MyDrive/BU/CS505/data_dump"
df_final = concat_csv(abs_path)

In [ ]:
df_final.head()

,post_title,post_date,post_link,post_content,post_content_type,post_upvote_count,post_comment_count,poster_karma,commenter_username,commenter_karma,comment_text,comment_upvotes,subreddit,post_date_extracted
0,Pro-Ukrainian Fighters Infiltrate Belgorod Reg...,2023-12-17 23:58:05,/r/worldnews/comments/18kv28m/proukrainian_fig...,https://www.kyivpost.com/post/25608,link,2185,52,-1,TheLordOfInfinity,-1,Sources in the Ukrainian intelligence communit...,182,NaN,2023-12-17
1,Pro-Ukrainian Fighters Infiltrate Belgorod Reg...,2023-12-17 23:58:05,/r/worldnews/comments/18kv28m/proukrainian_fig...,https://www.kyivpost.com/post/25608,link,2185,52,-1,CephalopodInstigator,956,"Not likely, this mob stirred some shit up in t...",154,NaN,2023-12-17
2,Pro-Ukrainian Fighters Infiltrate Belgorod Reg...,2023-12-17 23:58:05,/r/worldnews/comments/18kv28m/proukrainian_fig...,https://www.kyivpost.com/post/25608,link,2185,52,-1,bu11fr0g,36543,Russia needs to have pressure put on every one...,74,NaN,2023-12-17
3,North Korea fires another missile into sea in ...,2023-12-17 23:55:02,/r/worldnews/comments/18kv039/north_korea_fire...,https://apnews.com/article/north-korea-missile...,link,543,45,160494,JackC1126,57649,When will the sea respond to these constant at...,109,NaN,2023-12-17
4,North Korea fires another missile into sea in ...,2023-12-17 23:55:02,/r/worldnews/comments/18kv039/north_korea_fire...,https://apnews.com/article/north-korea-missile...,link,543,45,160494,throwawayacct420694,4281,"I’m other news, North Korea is out of food again,",56,NaN,2023-12-17


# Data cleaning

In [ ]:
import pandas as pd

abs_path = "/content/drive/MyDrive/BU/CS505/data_dump"
cleaned_file = "/combined_cleaned.csv"
cleaned_path = abs_path + cleaned_file
df = pd.read_csv(cleaned_path)

In [ ]:
df.head()

,post_title,subreddit,date,post_date,post_link,post_content,post_content_type,post_upvote_count,post_comment_count,poster_karma,commenter_username,commenter_karma,comment_text,comment_upvotes
0,Pro-Ukrainian Fighters Infiltrate Belgorod Reg...,worldnews,2023-12-17,2023-12-17 23:58:05,/r/worldnews/comments/18kv28m/proukrainian_fig...,https://www.kyivpost.com/post/25608,link,2185,52,-1,TheLordOfInfinity,-1,Sources in the Ukrainian intelligence communit...,182
1,Pro-Ukrainian Fighters Infiltrate Belgorod Reg...,worldnews,2023-12-17,2023-12-17 23:58:05,/r/worldnews/comments/18kv28m/proukrainian_fig...,https://www.kyivpost.com/post/25608,link,2185,52,-1,CephalopodInstigator,956,"Not likely, this mob stirred some shit up in t...",154
2,Pro-Ukrainian Fighters Infiltrate Belgorod Reg...,worldnews,2023-12-17,2023-12-17 23:58:05,/r/worldnews/comments/18kv28m/proukrainian_fig...,https://www.kyivpost.com/post/25608,link,2185,52,-1,bu11fr0g,36543,Russia needs to have pressure put on every one...,74
3,North Korea fires another missile into sea in ...,worldnews,2023-12-17,2023-12-17 23:55:02,/r/worldnews/comments/18kv039/north_korea_fire...,https://apnews.com/article/north-korea-missile...,link,543,45,160494,JackC1126,57649,When will the sea respond to these constant at...,109
4,North Korea fires another missile into sea in ...,worldnews,2023-12-17,2023-12-17 23:55:02,/r/worldnews/comments/18kv039/north_korea_fire...,https://apnews.com/article/north-korea-missile...,link,543,45,160494,throwawayacct420694,4281,"I’m other news, North Korea is out of food again,",56


In [ ]:
# df = df.drop(columns=["subreddit"])
# df = df.drop(columns=["post_date_extracted"])

In [ ]:
"/r/worldnews/comments/18kv28m/proukrainian_fig...	".split('/')

['', 'r', 'worldnews', 'comments', '18kv28m', 'proukrainian_fig...\t']

In [ ]:
def extract_subreddit(url):
    parts = url.split('/')
    if len(parts) > 1:
        return parts[2]
    else:
        return url

In [ ]:
def extract_exact_date(post_date):
    if len(post_date) < 10:
        return post_date
    date = post_date[0:10]
    return date

In [ ]:
df['subreddit'] = df['post_link'].apply(extract_subreddit)
df['date'] = df['post_date'].apply(extract_exact_date)

In [ ]:
column_order = ['post_title', 'subreddit', 'date'] + [col for col in df.columns if col not in ['post_title', 'subreddit', 'date']]
df = df[column_order]

In [ ]:
df.head()

,post_title,subreddit,date,post_date,post_link,post_content,post_content_type,post_upvote_count,post_comment_count,poster_karma,commenter_username,commenter_karma,comment_text,comment_upvotes
0,Pro-Ukrainian Fighters Infiltrate Belgorod Reg...,worldnews,2023-12-17,2023-12-17 23:58:05,/r/worldnews/comments/18kv28m/proukrainian_fig...,https://www.kyivpost.com/post/25608,link,2185,52,-1,TheLordOfInfinity,-1,Sources in the Ukrainian intelligence communit...,182
1,Pro-Ukrainian Fighters Infiltrate Belgorod Reg...,worldnews,2023-12-17,2023-12-17 23:58:05,/r/worldnews/comments/18kv28m/proukrainian_fig...,https://www.kyivpost.com/post/25608,link,2185,52,-1,CephalopodInstigator,956,"Not likely, this mob stirred some shit up in t...",154
2,Pro-Ukrainian Fighters Infiltrate Belgorod Reg...,worldnews,2023-12-17,2023-12-17 23:58:05,/r/worldnews/comments/18kv28m/proukrainian_fig...,https://www.kyivpost.com/post/25608,link,2185,52,-1,bu11fr0g,36543,Russia needs to have pressure put on every one...,74
3,North Korea fires another missile into sea in ...,worldnews,2023-12-17,2023-12-17 23:55:02,/r/worldnews/comments/18kv039/north_korea_fire...,https://apnews.com/article/north-korea-missile...,link,543,45,160494,JackC1126,57649,When will the sea respond to these constant at...,109
4,North Korea fires another missile into sea in ...,worldnews,2023-12-17,2023-12-17 23:55:02,/r/worldnews/comments/18kv039/north_korea_fire...,https://apnews.com/article/north-korea-missile...,link,543,45,160494,throwawayacct420694,4281,"I’m other news, North Korea is out of food again,",56


In [ ]:
df.groupby(['subreddit'])['post_title'].nunique()

subreddit
China             169
China_irl         427
news              116
politics          344
real_China_irl    413
worldnews         212
Name: post_title, dtype: int64

In [ ]:
df[df['subreddit']=='China_irl'].groupby(['post_content_type']).count()

,post_title,subreddit,date,post_date,post_link,post_content,post_upvote_count,post_comment_count,poster_karma,commenter_username,commenter_karma,comment_text,comment_upvotes
post_content_type,,,,,,,,,,,,,
image,402,402,402,402,402,402,402,402,402,402,402,402,402
link,870,870,870,870,870,870,870,870,870,870,870,870,870
text,2283,2283,2283,2283,2283,2283,2283,2283,2283,2283,2283,2283,2283
video,183,183,183,183,183,183,183,183,183,183,183,183,183


In [ ]:
df['post_title'].nunique()

1670

In [ ]:
df['date'].unique()

array(['2023-12-17', '2023-12-16', '2023-12-15', '2023-12-14',
       '2023-12-13', '2023-12-12', '2023-12-11'], dtype=object)

In [ ]:
file_path = abs_path + "/combined_cleaned.csv"
df.to_csv(file_path, encoding='utf-8-sig', index=False)

# Start from here

## OpenAI pip setup

In [33]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
!pip install --upgrade openai
!pip install PyPDF2
!pip install tiktoken
!pip install nltk
!pip install langchain

In [12]:
!pip install unstructured

## Preparing post dataframe and comment dataframe
### Post Only Dataframe

In [1]:
import pandas as pd
abs_path = "/content/drive/MyDrive/BU/CS505/data_dump"
file_path = "/combined_cleaned.csv"
combined_cleaned_path = abs_path + file_path
df = pd.read_csv(combined_cleaned_path)

In [2]:
df0 = df.copy()

In [3]:
df.columns

Index(['post_title', 'subreddit', 'date', 'post_date', 'post_link',
       'post_content', 'post_content_type', 'post_upvote_count',
       'post_comment_count', 'poster_karma', 'commenter_username',
       'commenter_karma', 'comment_text', 'comment_upvotes'],
      dtype='object')

In [22]:
df.head()

,post_title,subreddit,date,post_date,post_link,post_content,post_content_type,post_upvote_count,post_comment_count,poster_karma,commenter_username,commenter_karma,comment_text,comment_upvotes
0,Pro-Ukrainian Fighters Infiltrate Belgorod Reg...,worldnews,2023-12-17,2023-12-17 23:58:05,/r/worldnews/comments/18kv28m/proukrainian_fig...,https://www.kyivpost.com/post/25608,link,2185,52,-1,TheLordOfInfinity,-1,Sources in the Ukrainian intelligence communit...,182
1,Pro-Ukrainian Fighters Infiltrate Belgorod Reg...,worldnews,2023-12-17,2023-12-17 23:58:05,/r/worldnews/comments/18kv28m/proukrainian_fig...,https://www.kyivpost.com/post/25608,link,2185,52,-1,CephalopodInstigator,956,"Not likely, this mob stirred some shit up in t...",154
2,Pro-Ukrainian Fighters Infiltrate Belgorod Reg...,worldnews,2023-12-17,2023-12-17 23:58:05,/r/worldnews/comments/18kv28m/proukrainian_fig...,https://www.kyivpost.com/post/25608,link,2185,52,-1,bu11fr0g,36543,Russia needs to have pressure put on every one...,74
3,North Korea fires another missile into sea in ...,worldnews,2023-12-17,2023-12-17 23:55:02,/r/worldnews/comments/18kv039/north_korea_fire...,https://apnews.com/article/north-korea-missile...,link,543,45,160494,JackC1126,57649,When will the sea respond to these constant at...,109
4,North Korea fires another missile into sea in ...,worldnews,2023-12-17,2023-12-17 23:55:02,/r/worldnews/comments/18kv039/north_korea_fire...,https://apnews.com/article/north-korea-missile...,link,543,45,160494,throwawayacct420694,4281,"I’m other news, North Korea is out of food again,",56


In [4]:
posts_columns = ['post_title', 'subreddit', 'date', 'post_date', 'post_link',
                'post_content', 'post_content_type', 'post_upvote_count',
                'post_comment_count', 'poster_karma']
df_posts = df[posts_columns]
df_posts = df_posts.drop_duplicates()

In [5]:
df_posts.describe()

,post_upvote_count,post_comment_count,poster_karma
count,1683.000000,1683.000000,1.683000e+03
mean,833.593583,101.467023,2.186089e+05
std,2957.328263,337.432732,9.886928e+05
min,0.000000,1.000000,-1.000000e+00
25%,4.000000,8.000000,3.030000e+02
50%,33.000000,20.000000,4.200000e+03
75%,228.000000,61.000000,4.944000e+04
max,33946.000000,5234.000000,1.398153e+07


In [6]:
df_posts.to_csv(abs_path+'/reddit_post.csv', encoding='utf-8-sig', index=False)

### How does Assistant API handle urls?

In [27]:
df['post_content_type'].unique()

array(['link', 'text', 'image', 'video'], dtype=object)

In [28]:
df['subreddit'].unique()

array(['worldnews', 'news', 'politics', 'China', 'China_irl',
       'real_China_irl'], dtype=object)

In [31]:
# df_temp_1 = df.loc[(((df['post_content_type'] == 'link') | (df['post_content_type'] == 'text')) & (df['subreddit'] == 'worldnews'))]
df_link = df.loc[(df['post_content_type'] == 'link')]
col_link = ['post_title', 'post_content', 'post_content_type']
df_link = df_link[col_link]
df_link = df_link.drop_duplicates()
df_link['post_title'].count()

925

In [32]:
df_text = df.loc[(df['post_content_type'] == 'text')]
col_text = ['post_title', 'post_content', 'post_content_type']
df_text = df_text[col_text]
df_text = df_text.drop_duplicates()
df_text['post_title'].count()

477

## OpenAI API Calls

In [34]:
import openai
import tiktoken
import requests
import tarfile
import PyPDF2
import pandas as pd
from io import BytesIO
from PyPDF2 import PdfReader
import time

In [35]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.docstore.document import Document
from unstructured.cleaners.core import remove_punctuation,clean,clean_extra_whitespace
from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain

In [36]:
# OpenAI API Key
api_path = '/content/drive/MyDrive/Colab Notebooks/openai_apikey.txt'
API_KEY = ""
with open(api_path, 'r') as file:
    API_KEY = file.readline().strip()

In [37]:
import os

os.environ['OPENAI_API_KEY'] = API_KEY

In [39]:
def generate_document(url):
    "Given an URL, return a langchain Document to futher processing"
    loader = UnstructuredURLLoader(urls=[url],
    mode="elements",
    post_processors=[clean,remove_punctuation,clean_extra_whitespace])
    elements = loader.load()
    selected_elements = [e for e in elements if e.metadata['category']=="NarrativeText"]
    full_clean = " ".join([e.page_content for e in selected_elements])

    return Document(page_content=full_clean, metadata={"source":url})

In [40]:
def summarize_document(url,model_name='gpt-3.5-turbo-instruct'):
    "Given an URL return the summary from OpenAI model"
    llm = OpenAI(model_name=model_name,temperature=0,openai_api_key=API_KEY)
    chain = load_summarize_chain(llm, chain_type="stuff")
    tmp_doc = generate_document(url)
    summary = chain.run([tmp_doc])

    return clean_extra_whitespace(summary)

In [62]:
def toxicity_score():
    "TBD"
    return

In [110]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)
temp =
print(completion.choices[0].message.content)

str

In [116]:
from openai import OpenAI

def toxicity_score(title, content, MODEL):
    "Given a post's title and summary, evaluate the toxicity from 0 to 10"
    client = OpenAI()
    openai.api_key = API_KEY
    SYSTEM_PROMPT = "You are an expert fact-checker that identifies toxicity."
    USER_PROMPT = "Please evaluate the following content for its level of toxicity. \
                    Provide a rating on a scale from 0 to 10, where 0 indicates no toxicity \
                    (completely neutral or positive content), and 10 indicates extremely \
                    toxic content, such as content that is harmful, offensive, or incites \
                    negativity. Base your assessment on the tone, language, and potential \
                    impact of the content. \
                    Do not write anything but a 0-10 score \n\n \
                    Reddit Post Title:\n\n{}\
                    Reddit Post Content:\n\n{}".format(title, content)
    TEMPERATURE_LIMIT = 0
    TOKEN_LIMIT = 512

    completion = client.chat.completions.create(
        model=MODEL,
        messages=[
            # First insert System prompt
            {"role": "system", "content": SYSTEM_PROMPT},
            # Second prompt the User Prompt
            {"role": "user", "content": USER_PROMPT},
        ],
        temperature=TEMPERATURE_LIMIT,
        max_tokens=TOKEN_LIMIT,
    )

    api_response_content = completion.choices[0].message.content

    return api_response_content

In [45]:
df_link.head()

,post_title,post_content,post_content_type
0,Pro-Ukrainian Fighters Infiltrate Belgorod Reg...,https://www.kyivpost.com/post/25608,link
3,North Korea fires another missile into sea in ...,https://apnews.com/article/north-korea-missile...,link
6,Germany: Far-right AfD wins first city mayoral...,https://www.dw.com/en/germany-far-right-afd-wi...,link
9,Serbia's Vucic claims big election victory for...,https://www.bbc.co.uk/news/world-europe-67742032,link
12,Ukraine-based Russian paramilitaries claim cro...,https://www.reuters.com/world/europe/ukraine-b...,link


In [57]:
df_tmp = df_link[df_link.index > 1824]
df_tmp.head()

,post_title,post_content,post_content_type
1827,"Nikki Haley Is an Anti-Worker Fanatic, Not a “...",https://jacobin.com/2023/12/nikki-haley-republ...,link
1830,Cable lobby and Republicans fight proposed ban...,https://arstechnica.com/tech-policy/2023/12/fc...,link
1833,"2024: Stopping Trump, Renewing Democracy",https://plus.thebulwark.com/p/2024-stopping-tr...,link
1836,New Colorado law aims to give some incarcerate...,https://coloradosun.com/2023/12/12/parenting-f...,link
1839,Republicans Tap Israeli Military Veteran to Ru...,https://www.nytimes.com/2023/12/14/nyregion/pi...,link


In [58]:
# Summarize Link Posts

import time
import datetime

df_return = pd.DataFrame()
model = 'gpt-3.5-turbo-instruct'

# Summarize all links
for index, row in df_tmp.iterrows():
    retries = 0
    results = []

    while retries < 2:
        print("Processing index {}".format(index))
        try:
            summary = summarize_document(row['post_content'], model)
            df_results = row.to_frame().transpose()
            df_results['summary'] = summary
            path = abs_path + '/link_summary_' + str(index) + '.csv'
            df_results.to_csv(path, index=True)
            break

        except Exception as e:
            print("Error Message: {}".format(e))

        retries += 1

    df_return = pd.concat([df_return, df_results])

Processing index 1827
Processing index 1830
Processing index 1833
Processing index 1836
Processing index 1839
Processing index 1842
Processing index 1845
Processing index 1848
Processing index 1851
Processing index 1854
Processing index 1857
Processing index 1860
Processing index 1863
Processing index 1866
Processing index 1869
Processing index 1872
Processing index 1875
Processing index 1878
Processing index 1881
Processing index 1884
Processing index 1887
Processing index 1890
Processing index 1893
Processing index 1896
Processing index 1899
Processing index 1902
Processing index 1905
Processing index 1908
Processing index 1911
Processing index 1914
Processing index 1917
Processing index 1920
Error Message: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 5872 tokens (5616 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': 

ERROR:langchain_community.document_loaders.url:Error fetching or processing /r/Wechat/comments/18jq08s/info_on_what_can_get_you_banned_on_wechat/, exception: Invalid URL '/r/Wechat/comments/18jq08s/info_on_what_can_get_you_banned_on_wechat/': No scheme supplied. Perhaps you meant https:///r/Wechat/comments/18jq08s/info_on_what_can_get_you_banned_on_wechat/?


Processing index 2068
Processing index 2073
Processing index 2074
Processing index 2080
Processing index 2095
Processing index 2098
Processing index 2104
Processing index 2110
Processing index 2124
Processing index 2127
Processing index 2130
Processing index 2136
Processing index 2138
Processing index 2141
Processing index 2144
Processing index 2147
Error Message: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 4102 tokens (3846 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Processing index 2147
Error Message: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 4102 tokens (3846 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Processing index 216

ERROR:langchain_community.document_loaders.url:Error fetching or processing https://news.futunn.com/en/post/35129633?level=2&data_ticket=1702582578111968, exception: Detected a JSON file that does not conform to the Unstructured schema. partition_json currently only processes serialized Unstructured output.


Processing index 2182
Processing index 2191
Processing index 2205
Processing index 2211
Processing index 2235
Processing index 2238
Processing index 2244
Processing index 2247


...

Processing index 3899
Processing index 3906
Processing index 3920
Processing index 3929
Processing index 3938
Processing index 3970
Processing index 3979
Processing index 4007
Processing index 4028
Processing index 4049
Processing index 4072
Processing index 4085


ERROR:langchain_community.document_loaders.url:Error fetching or processing https://i.imgur.com/SyJMfi3.jpg, exception: name 'partition_image' is not defined


Processing index 4091
Processing index 4106
Processing index 4117
Processing index 4120
Processing index 4123
Processing index 4152
Processing index 4183
Processing index 4192
Processing index 4195
Processing index 4198
Processing index 4201
Processing index 4210
Processing index 4213
Processing index 4219
Processing index 4234
Processing index 4250
Processing index 4289
Processing index 4305
Processing index 4314


ERROR:langchain_community.document_loaders.url:Error fetching or processing /r/iwanttorun/comments/18ivldr/华为工作经历老婆党员加拿大pr申请安调概率和时间/, exception: Invalid URL '/r/iwanttorun/comments/18ivldr/华为工作经历老婆党员加拿大pr申请安调概率和时间/': No scheme supplied. Perhaps you meant https:///r/iwanttorun/comments/18ivldr/华为工作经历老婆党员加拿大pr申请安调概率和时间/?


Processing index 4317
Processing index 4320
Processing index 4341
Processing index 4345
Processing index 4346
Processing index 4389
Processing index 4392
Processing index 4407
Processing index 4419
Processing index 4425
Processing index 4431
Processing index 4434
Processing index 4443
Processing index 4461
Processing index 4518
Processing index 4556
Processing index 4588
Processing index 4594
Processing index 4597
Processing index 4633
Processing index 4639
Processing index 4668
Processing index 4683
Processing index 4692
Processing index 4701
Processing index 4762
Processing index 4796
Processing index 4840


In [69]:
import glob

def combine_csv_files(file_pattern, output_file):
    """
    Combine all CSV files matching the given pattern into one DataFrame
    and save it to a new CSV file with utf-8-sig encoding.

    Returns combined csv file.
    """
    files = glob.glob(file_pattern)

    # Initialize an empty list to store DataFrames
    dataframes = []

    # Iterate over the files and read each into a DataFrame
    for file in files:
        df = pd.read_csv(file, index_col=0)  # Ensure the first column is used as the index
        dataframes.append(df)

    # Combine all DataFrames into one
    link_summary_combined = pd.concat(dataframes)

    # Save the combined DataFrame to a new CSV file with utf-8-sig encoding
    link_summary_combined.to_csv(output_file, encoding='utf-8-sig')

    return link_summary_combined

# Usage
abs_path = '/content/drive/MyDrive/BU/CS505/data_dump'
link_summary_csv_path = abs_path + '/link_summary_*.csv'
combined_link_summary_path = abs_path + '/combined_link_summary.csv'
df_link_summary_combined = combine_csv_files(link_summary_csv_path, combined_link_summary_path)

## Data Cleaning for Link Posts summarized by Web Agents
Of all the link posts, a lot are rejected or did not go through。

In [94]:
df_link.nunique()

post_title           923
post_content         914
post_content_type      1
dtype: int64

In [79]:
df_link_summary_combined = pd.read_csv(combined_link_summary_path, index_col=0)

In [80]:
df_link_summary_combined.head()

,post_title,post_content,post_content_type,summary
0,Pro-Ukrainian Fighters Infiltrate Belgorod Reg...,https://www.kyivpost.com/post/25608,link,A shooting battle is taking place on the borde...
3,North Korea fires another missile into sea in ...,https://apnews.com/article/north-korea-missile...,link,North Korea has released images of their lates...
6,Germany: Far-right AfD wins first city mayoral...,https://www.dw.com/en/germany-far-right-afd-wi...,link,"Tim Lochner, an independent candidate running ..."
9,Serbia's Vucic claims big election victory for...,https://www.bbc.co.uk/news/world-europe-67742032,link,Serbian President Aleksandar Vucic has claimed...
12,Ukraine-based Russian paramilitaries claim cro...,https://www.reuters.com/world/europe/ukraine-b...,link,There is no information provided to summarize.


In [90]:
import pandas as pd

# Get the counts of unique values in the 'summary' column
summary_counts = df_link_summary_combined['summary'].value_counts()

# Extract the top 10 most common texts along with their counts
top_20_common_summaries = summary_counts.head(20)

print(top_20_common_summaries)

Your request has been blocked due to a network policy. Please log in or create an account to continue browsing. If you are using a script or application, make sure to register with your developer credentials and use a unique and descriptive User-Agent. If you believe you have been incorrectly blocked or need assistance, please file a ticket and include your IP address and Reddit account information.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [91]:
# Count the occurrences of each summary
summary_counts = df_link_summary_combined['summary'].value_counts()

# Find the summaries that occur only once
unique_summaries = summary_counts[summary_counts == 1].index

# Keep only the rows where the summary is unique
df_link_summary_combined_unique = df_link_summary_combined[df_link_summary_combined['summary'].isin(unique_summaries)]

In [92]:
df_link_summary_combined_unique.nunique()

post_title           539
post_content         539
post_content_type      1
summary              540
dtype: int64

## Combine link posts and text posts, also copy post_content into summary for text based posts.

In [96]:
df_text_summary_combined_unique = df_text.copy()
df_text_summary_combined_unique['summary'] = df_text_summary_combined_unique['post_content']

In [97]:
df_text_summary_combined_unique.nunique()

post_title           476
post_content         469
post_content_type      1
summary              469
dtype: int64

In [98]:
df_posts_summary = pd.concat([df_link_summary_combined_unique, df_text_summary_combined_unique])

In [100]:
df_posts_summary.sample(10)

,post_title,post_content,post_content_type,summary
2352,The issuance of visas to Chinese citizens by t...,Tourist visas in October/ student visas in Ju...,text,Tourist visas in October/ student visas in Ju...
2675,如果中国结束大一统，分裂成多个国家,那么要如何分裂才能确保每一个分裂出来的小国都有条件发展成为发达国家？\n\n新疆靠石油，沪国...,text,那么要如何分裂才能确保每一个分裂出来的小国都有条件发展成为发达国家？\n\n新疆靠石油，沪国...
516,"Indian Navy sends warship, aircraft to assist ...",https://www.thehindubusinessline.com/news/indi...,link,The Indian Navy has responded quickly to the h...
421,North Korean nuclear attack would end Kim's re...,https://www.jpost.com/breaking-news/article-77...,link,The US and South Korea have warned North Korea...
208,Florida Republican Party suspends chairman and...,https://www.theglobeandmail.com/world/article-...,link,The Republican Party of Florida suspended chai...
1059,Firefighters search for anyone trapped after c...,https://apnews.com/article/bronx-building-coll...,link,A six-story corner of a Bronx apartment buildi...
1991,"""Pure"" or ""Real"" Chinese. What does this mean?","To me, you are Chinese if you have a Chinese p...",text,"To me, you are Chinese if you have a Chinese p..."
640,Switzerland takes step closer to screening for...,https://www.swissinfo.ch/eng/business/switzerl...,link,The Swiss government has drafted legislation t...
3447,有没有弯弯来解释下台湾出了很多恐怖游戏？,https://store.steampowered.com/app/1611430/_/,link,"""The game ""The Bridge Curse Road to Salvation""..."
2033,What do the Chinese people do in their free time?,"What are some popular hobbies? I usually read,...",text,"What are some popular hobbies? I usually read,..."


## Calculate Toxicity and Data Cleaning

In [119]:
import time
import datetime

df_posts_summary_return = pd.DataFrame()
model = 'gpt-4-1106-preview'

# Summarize all links
for index, row in df_posts_summary.iterrows():
    retries = 0
    results = []

    while retries < 2:
        print("Processing index {}".format(index))
        try:
            score = toxicity_score(title=row['post_title'], content=row['summary'], MODEL=model)
            df_results = row.to_frame().transpose()
            df_results['score'] = score
            # path = abs_path + '/link_summary_' + str(index) + '.csv'
            # df_results.to_csv(path, index=True)
            break

        except Exception as e:
            print("Error Message: {}".format(e))

        retries += 1

    df_posts_summary_return = pd.concat([df_posts_summary_return, df_results])

Processing index 0
Processing index 3
Processing index 6
Processing index 9
Processing index 19
Processing index 21
Processing index 24
Processing index 27
Processing index 36
Processing index 48
Processing index 54
Processing index 57
Processing index 63
Processing index 66
Processing index 69
Processing index 72
Processing index 78
Processing index 84
Processing index 90
Processing index 93
Processing index 96
Processing index 102
Processing index 108
Processing index 110
Processing index 111
Processing index 117
Processing index 120
Processing index 126
Processing index 129
Processing index 136
Processing index 139
Processing index 142
Processing index 145
Processing index 154
Processing index 157
Processing index 166
Processing index 175
Processing index 178
Processing index 181
Processing index 184
Processing index 187
Processing index 190
Processing index 193
Processing index 199
Processing index 205
Processing index 208
Processing index 211
Processing index 214
Processing index 

In [125]:
def is_int_in_range(value):
    try:
        num = int(value)
        return 0 <= num <= 10
    except ValueError:
        return False

# Apply the function to the column and keep rows where the condition is True
df_posts_summary_return = df_posts_summary_return[df_posts_summary_return['score'].apply(is_int_in_range)]

In [126]:
df_posts_summary_return['score'].value_counts()

0     626
2     155
1     126
5      33
8      29
6      18
3      10
7       7
10      4
9       3
4       2
Name: score, dtype: int64

In [129]:
posts_summary_return_path = abs_path + '/posts_summary_return.csv'
df_posts_summary_return.to_csv(posts_summary_return_path)

In [130]:
df_posts.head()

,post_title,subreddit,date,post_date,post_link,post_content,post_content_type,post_upvote_count,post_comment_count,poster_karma
0,Pro-Ukrainian Fighters Infiltrate Belgorod Reg...,worldnews,2023-12-17,2023-12-17 23:58:05,/r/worldnews/comments/18kv28m/proukrainian_fig...,https://www.kyivpost.com/post/25608,link,2185,52,-1
3,North Korea fires another missile into sea in ...,worldnews,2023-12-17,2023-12-17 23:55:02,/r/worldnews/comments/18kv039/north_korea_fire...,https://apnews.com/article/north-korea-missile...,link,543,45,160494
6,Germany: Far-right AfD wins first city mayoral...,worldnews,2023-12-17,2023-12-17 23:42:20,/r/worldnews/comments/18kuqkf/germany_farright...,https://www.dw.com/en/germany-far-right-afd-wi...,link,1974,400,8062862
9,Serbia's Vucic claims big election victory for...,worldnews,2023-12-17,2023-12-17 23:24:19,/r/worldnews/comments/18kucfd/serbias_vucic_cl...,https://www.bbc.co.uk/news/world-europe-67742032,link,31,7,-1
12,Ukraine-based Russian paramilitaries claim cro...,worldnews,2023-12-17,2023-12-17 23:02:03,/r/worldnews/comments/18ktush/ukrainebased_rus...,https://www.reuters.com/world/europe/ukraine-b...,link,209,1,107645


In [131]:
df_posts_summary_return.head()

,post_title,post_content,post_content_type,summary,score
0,Pro-Ukrainian Fighters Infiltrate Belgorod Reg...,https://www.kyivpost.com/post/25608,link,A shooting battle is taking place on the borde...,0
3,North Korea fires another missile into sea in ...,https://apnews.com/article/north-korea-missile...,link,North Korea has released images of their lates...,0
6,Germany: Far-right AfD wins first city mayoral...,https://www.dw.com/en/germany-far-right-afd-wi...,link,"Tim Lochner, an independent candidate running ...",1
9,Serbia's Vucic claims big election victory for...,https://www.bbc.co.uk/news/world-europe-67742032,link,Serbian President Aleksandar Vucic has claimed...,0
19,David Cameron calls on Hong Kong to release Ji...,https://www.theguardian.com/world/2023/dec/17/...,link,"Jimmy Lai, a pro-democracy newspaper publisher...",1


In [132]:
col_temp = ["subreddit","date","post_date","post_link","post_upvote_count","post_comment_count","poster_karma"]
df_posts_final = df_posts_summary_return\
                .merge(df_posts[col_temp], left_index=True, right_index=True, how='left')

In [133]:
posts_final_path = abs_path + '/posts_final.csv'
df_posts_final.to_csv(posts_final_path)

# Limitations
We still want to build misinformation Agent and also examine the comment's allignment to the post's sentiment.